### 1. General info of dataset GSE235063

This is the Jupyter Notebook for dataset GSE235063. Its dataset includes barcodes/genes/matrix files for each sample.

Thus, we need to simply incorparate these barcodes/genes/matrix files and generate an AnnData object for each sample. 

In total, there are 75 acute myeloid leukemia (AML) samples. The datasets includes both raw and processed information (thus, a total of 150 samples).

<span style="color:green">**[DX]**</span> samples from diagnosis

<span style="color:green">**[REM]**</span> samples from remission

<span style="color:green">**[REL]**</span> samples from relapse 

In [9]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy
import os

import warnings

# Ignore all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 2. AnnData object of each sample

<span style="color:red">**IMPORTANT:**</span> rename files to get rid of prefixes

1. `barcodes.tsv`: cell barcodes, which go into `.obs`
2. `genes.tsv`: gene names, `.var`
3. `matrix.mtx`: the expression matrix, `.X`

<span style="color:red">**Problem:**</span> the genes.tsv files from the processed dataset have MISSING gene identifier (such as ENSG00000268674) information --> some ensenbl gene IDs are pointing to the same gene symbol in the complete genes.tsv files from the raw dataset, so it's ambiguous to assign an appropriate gene IDs to these gene symbols --> duplicate the columns in the processed genes.tsv files from the processed dataset

The processed dataset includes metadata (patient age, biopsy origin, etc) for each sample. So we need to add information from metadata to each sample AnnData obejct.

In [10]:
from pathlib import Path

# Specify directory paths
data_directory = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE235063')
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE235063')

for sample_directory in data_directory.iterdir():
    sample_name = sample_directory.stem
    sample_h5ad = sample_name + '_uni.h5ad'

    if 'processed' not in sample_name:
        pro_sample_name = sample_name.split("raw")[0] + "processed" # the corresponding processed sample
        metadata_path = data_directory / pro_sample_name / "metadata.tsv" 
    else:
        # Duplicate the column in genes.tsv
        file_path = sample_directory / 'genes.tsv'
        file_df = pd.read_csv(file_path, sep='\t', header=None, names=['gene_identifier'])
        file_df['gene_symbols'] = file_df['gene_identifier']
        file_df.to_csv(file_path, sep='\t', index=False, header=False)

        metadata_path = data_directory / sample_name / "metadata.tsv"

    anndata = sc.read_10x_mtx(
        sample_directory,
        var_names='gene_symbols',  
        cache=False
    )

    # Create an observation metric info to store related features
    obs_metrics = pd.DataFrame(index=anndata.obs_names) ## Get the identifiers
    obs_metrics['cancer_type'] = 'acute_myeloid_leukemia'
    obs_metrics['dataset'] = 'GSE235063'
    obs_metrics['sample_barcode'] = sample_name
    obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
    
    # Load the metadata as a DataFrame
    metadata = pd.read_csv(metadata_path, sep="\t", index_col="Cell_Barcode")
    # Merge columns from the metadata to the obs_metrics
    cols_to_merge = ["Patient_Sample", "Classified_Celltype", "Malignant", "Biopsy_Origin", "Age_Months"]
    obs_metrics_merged = obs_metrics.merge(metadata[cols_to_merge], how="left", left_index=True, right_index=True)

    # Fill the null with correct info
    cols = ["Patient_Sample", "Biopsy_Origin", "Age_Months"]
    for col in cols:
        obs_metrics_merged[col].fillna(metadata[col].unique()[0], inplace=True)

    # Rename the columns for consistency
    obs_metrics_merged.rename(columns={"Patient_Sample": "disease_progression", "Classified_Celltype": "celltype_from_paper",
                                   "Malignant": "malignant_from_paper", "Biopsy_Origin": "tissue", "Age_Months": "age_months"},
                                   inplace=True)

    anndata.obs = obs_metrics_merged
    anndata.obs.set_index("uni_barcode", drop=False, inplace=True)

    # save the anndata object
    anndata.write_h5ad(write_directory / sample_h5ad, compression="gzip")

In [11]:
anndata

AnnData object with n_obs × n_vars = 6794880 × 33538
    obs: 'cancer_type', 'dataset', 'sample_barcode', 'uni_barcode', 'disease_progression', 'celltype_from_paper', 'malignant_from_paper', 'tissue', 'age_months'
    var: 'gene_ids'

In [13]:
anndata.obs

,cancer_type,dataset,sample_barcode,uni_barcode,disease_progression,celltype_from_paper,malignant_from_paper,tissue,age_months
uni_barcode,,,,,,,,,
GSE235063_AAACCCAAGAAACACT-1,acute_myeloid_leukemia,GSE235063,GSM7494260_AML6_DX_raw,GSE235063_AAACCCAAGAAACACT-1,Diagnosis,NaN,NaN,Blood,90.0
GSE235063_AAACCCAAGAAACCAT-1,acute_myeloid_leukemia,GSE235063,GSM7494260_AML6_DX_raw,GSE235063_AAACCCAAGAAACCAT-1,Diagnosis,NaN,NaN,Blood,90.0
GSE235063_AAACCCAAGAAACCCA-1,acute_myeloid_leukemia,GSE235063,GSM7494260_AML6_DX_raw,GSE235063_AAACCCAAGAAACCCA-1,Diagnosis,NaN,NaN,Blood,90.0
GSE235063_AAACCCAAGAAACCCG-1,acute_myeloid_leukemia,GSE235063,GSM7494260_AML6_DX_raw,GSE235063_AAACCCAAGAAACCCG-1,Diagnosis,NaN,NaN,Blood,90.0
GSE235063_AAACCCAAGAAACCTG-1,acute_myeloid_leukemia,GSE235063,GSM7494260_AML6_DX_raw,GSE235063_AAACCCAAGAAACCTG-1,Diagnosis,NaN,NaN,Blood,90.0
...,...,...,...,...,...,...,...,...,...
GSE235063_TTTGTTGTCTTTGCTA-1,acute_myeloid_leukemia,GSE235063,GSM7494260_AML6_DX_raw,GSE235063_TTTGTTGTCTTTGCTA-1,Diagnosis,NaN,NaN,Blood,90.0
GSE235063_TTTGTTGTCTTTGCTG-1,acute_myeloid_leukemia,GSE235063,GSM7494260_AML6_DX_raw,GSE235063_TTTGTTGTCTTTGCTG-1,Diagnosis,NaN,NaN,Blood,90.0
GSE235063_TTTGTTGTCTTTGGAG-1,acute_myeloid_leukemia,GSE235063,GSM7494260_AML6_DX_raw,GSE235063_TTTGTTGTCTTTGGAG-1,Diagnosis,NaN,NaN,Blood,90.0


### 3. Convert AnnData objects to SingleCellExperiment objects

In [17]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE235063')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if "_uni.h5ad" in file.name:
        sample_anndata = sc.read_h5ad(file)
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        # print(sample_sce)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))